In [1]:
# play audio in ipython notebook
try:
    import winsound
except ImportError:
    !pip install winsound
    import winsound
duration = 1000  # milliseconds
freq = 440  # Hz

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

In [2]:
import numpy as np
import time
import requests
import os
import pickle
from lxml import html
import numpy as np
import urllib.request
import face_recognition
import cv2
import matplotlib.pyplot as plt
import pytesseract
from googlesearch import search
from PIL import Image, ImageStat
import time
tt = time.time
st = tt()

time: 3.14 s


In [3]:
text = 'blah blah prof first middle last'.split(' ')
idx = text.index('prof')
text = text[idx:]
text[:3]

['prof', 'first', 'middle']

time: 4.12 ms


In [4]:
# get prof name from thumbnail
def save_as_pickle(file_path, file):
    with open(file_path, 'wb') as f:
        pickle.dump(file, f)

def load_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        file = pickle.load(f)
    return file

def plot_image_list(img_list, figsize=(12, 12), subplot_n_cols=2):
    plt.figure(figsize=(12, 12))
    for i,img in enumerate(img_list):
        plt.subplot(int(len(img_list)/subplot_n_cols)+1, subplot_n_cols, i+1)
        try:
            plt.imshow(img)
        except:
            pass
        
def make_rgb_bgr(img):
    b,g,r = cv2.split(img)          
    rgb_img = cv2.merge([r,g,b]) 
    return rgb_img
        
def brightness(img):
    img = Image.fromarray(img)
    img = img.convert('L')
    stat = ImageStat.Stat(img)
    return stat.rms[0]

def is_valid_img(img, lthresh = 100, hthresh = 200):
    bright = brightness(img)
    if bright<lthresh or bright>hthresh:
        return False
    else:
        return True
        
def get_extension_files(path_to_json, ext='.json'):
    json_files = [os.path.join(path_to_json, pos_json) for pos_json in os.listdir(path_to_json) if pos_json.endswith(ext)]
    return json_files

def flatten_list(l):
    return [item for sublist in l for item in sublist]

def get_prof_name(text, title=False):
    hon_list = ['prof','dr', 'by']
    ext_list = ['.', ' ']
    hon_list = flatten_list([[w+ext for ext in ext_list] for w in hon_list])
    
    word_before_list = ['assosiate', 'adjunct', 'professor', 'faculty', 'dept.', 'department']
    ishon=False
    text = text.split(' ')
    text_copy = text.copy()
    for hon in hon_list:
        if hon in text:       
            print(f" honorific in text: {text_copy}")
            idx = text.index(hon)
            text = text[idx:]
            if title:
                text = " ".join(text)
                return text
                
            else:
                # get all words before a specified word
                for wb in word_before_list:
                    if wb in text:
                        wbidx = text.index(wb)
                        text = " ".join(text[:wbidx])
                        text = ''.join([x for x in text if x.isalpha() or x in ext_list])  # get rid of non apha chars
                        print(f" honorific and word_brfore in text: {text_copy}")
                        return text
                
                # if last name is less than 4 letters dont include
                if len(text)>3:
                    if len(text[3])<4:
                        text = text[:3]
                    else:
                        text = text[:4]
                return " ".join(text)
            
    return None
    
def get_name_fom_image(img):
    
    tesseract_file_path = "C:/Users/YASH/AppData/Local/Tesseract-OCR/tesseract.exe"
    pytesseract.pytesseract.tesseract_cmd = tesseract_file_path
#     img = subimg(img, u=0.7)

    # Convert the image to gray scale 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

    # Performing OTSU threshold 
    ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY) 

    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18)) 

    # Appplying dilation on the threshold image 
    dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1) 
    # dilation = thresh1

    # Finding contours 
    contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,  
                                                     cv2.CHAIN_APPROX_NONE) 

    # Creating a copy of image 
    im2 = img.copy() 
    text = None
    for cnt in contours: 
        x, y, w, h = cv2.boundingRect(cnt) 

        # Drawing a rectangle on copied image 
        rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2) 

        # Cropping the text block for giving input to OCR 
        cropped = im2[y:y + h, x:x + w] 

        # Apply OCR on the cropped image 
        text_sub = pytesseract.image_to_string(cropped)
        text_sub = text_sub.lower().replace("\n"," ").strip()
        if text_sub=='':
            continue
        text_sub_name = get_prof_name(text_sub)
        if text_sub_name is not None:
            print(text_sub)
            text = text_sub_name
            break
    
    return text

def post_processing(img):
    norm_img = np.zeros((img.shape[0], img.shape[1]))
    norm_img = cv2.normalize(img, norm_img, 0, 255,
                             cv2.NORM_MINMAX)
    return img


time: 11 ms


In [5]:
# %%capture cap --no-stderr

# # get NAMES ONLY

# base_dir = 'C:\\Users\\YASH\\nptel_face_recognition'

# full_vedios_dir = os.path.join(base_dir, "full_vedio")
# known_faces_dir = os.path.join(base_dir, "known_faces")
# tracebacks_dir = os.path.join(base_dir, "tracebacks")
# os.makedirs(tracebacks_dir, exist_ok = True)

# pkl_files_dir = os.path.join(base_dir, "pkl_files")
# nptel_channel_name_list = load_from_pickle(os.path.join(pkl_files_dir, "nptel_channel_name_list.pkl"))
# # num_channels = 2
# # num_playlists = 2
# # num_vedios = 2
    
# num_channels = 5
# num_playlists = 5
# num_vedios = 5

# playlist_2_prof_name_dict={}
# # for nptel_channel_name in nptel_channel_name_list:
# for nptel_channel_name in nptel_channel_name_list[:num_channels]:
#     print(f'\n\n\nfor nptel_channel_name {nptel_channel_name}')
#     chanel_vedio_dir = os.path.join(full_vedios_dir, nptel_channel_name)
#     playlist_path_list = [x[0] for x in os.walk(chanel_vedio_dir)][1:]

#     print(f'number of playlist: {len(playlist_path_list)}')
#     vedio_ext = '.mp4'
    
#     namef=0
#     namenf=0
#     namef_playlist_names_list=[]
#     namenf_playlist_names_list=[]
#     for playlist_path in playlist_path_list[:num_playlists]:
#         gotname = False
#         playlistn = playlist_path.split('\\')[-1]
#         print(f"\npaylist: {playlistn}")
#         vedio_path_list = get_extension_files(playlist_path, ext=vedio_ext)
#         print(f"number of vedios: {len(vedio_path_list)}")
        
#         vst = 5
#         search_interval = 20
#         vet = vst + search_interval

#         num_f_per_sec = 1
#         num_f_inc = 3
        

#         text_sub = get_prof_name(playlistn.lower(), title=True)
#         if text_sub is not None:
#             print(f'name: {text_sub} in the title')
#             prof_name = text_sub 
#             gotname = True

#         for vedio_path in vedio_path_list[:num_vedios]:
#             if gotname:
#                 continue
#             else:
#                 num_f_per_sec+=num_f_inc

#             vfname = vedio_path.split('\\')[-1]
#             print(f"vedio: {vfname}")
#             print(f"(vst, vet, num_f_per_sec): {(vst, vet, num_f_per_sec)}")
#             video_capture = cv2.VideoCapture(vedio_path)
#             fps = int(video_capture.get(cv2.CAP_PROP_FPS))
#             st = tt()
#             i=-1
#             while video_capture.isOpened():
#                 i+=1
#                 if int(i/fps) < vst:
#                     ret, frame = video_capture.read()
#                     continue

#                 if int(i/fps) > vet:
#                     break

#                 ret, frame = video_capture.read()

#                 if i%int(fps/num_f_per_sec)==0:
#     #                 print(f"time(sec): {int(i/fps)}")
#                     if not gotname:
#                         text = get_name_fom_image(frame)
#                         if text is not None:
#                             print(f'name: {text} at time: {int(i/fps)}')
#                             prof_name = text
#                             gotname = True
#                             break

#                 if cv2.waitKey(1) & 0xFF == ord('q'):
#                     break

#             video_capture.release()
#             cv2.destroyAllWindows()

#         if gotname:
#             print(f"face not found for {prof_name}")
#             namef+=1
#             namef_playlist_names_list.append(playlistn)
#             playlist_2_prof_name_dict[playlistn] = prof_name
#         else:
#             print(f"face and name not found for {playlist_path}")
#             namenf+=1
#             playlist_2_prof_name_dict[playlistn] = None
#             playlist_2_prof_name_dict.append(playlistn)
#             namenf+=1
    
#     chanel_pkl_files_dir = os.path.join(pkl_files_dir, nptel_channel_name)
#     save_as_pickle(os.path.join(chanel_pkl_files_dir, "namef_playlist_names_list.pkl"), namef_playlist_names_list)
#     save_as_pickle(os.path.join(chanel_pkl_files_dir, "namenf_playlist_names_list.pkl"), namenf_playlist_names_list)
#     print(f"\nfor channel: {nptel_channel_name}  names found: {namef}, names not found: {namenf}")    

# prof_name_2_playlist_dict = {v: k for k, v in playlist_2_prof_name_dict.items()}
# save_as_pickle(os.path.join(pkl_files_dir, "prof_name_2_playlist_dict.pkl"), prof_name_2_playlist_dict)
# save_as_pickle(os.path.join(pkl_files_dir, "playlist_2_prof_name_dict.pkl"), playlist_2_prof_name_dict)
    
# winsound.Beep(freq, duration)

NameError: name 'cap' is not defined

time: 14min 3s


In [ ]:
# with open(os.path.join(tracebacks_dir, "prof_names.txt"), 'w', encoding='utf-8') as f:
#     f.write(cap.stdout)

In [25]:
%%capture cap --no-stderr

# save faces and names

base_dir = 'C:\\Users\\YASH\\nptel_face_recognition'

full_vedios_dir = os.path.join(base_dir, "full_vedio")
known_faces_dir = os.path.join(base_dir, "known_faces")
tracebacks_dir = os.path.join(base_dir, "tracebacks")
os.makedirs(tracebacks_dir, exist_ok = True)

pkl_files_dir = os.path.join(base_dir, "pkl_files")
nptel_channel_name_list = load_from_pickle(os.path.join(pkl_files_dir, "nptel_channel_name_list.pkl"))
# num_channels = 2
# num_playlists = 2
# num_vedios = 2

num_channels = 5
num_playlists = 5
num_vedios = 5

playlist_2_prof_name_dict={}
tfacef=0
tnamef=0
tnamenf=0
# for nptel_channel_name in nptel_channel_name_list:
for nptel_channel_name in nptel_channel_name_list[:num_channels]:
    print(f'\n\n\nfor nptel_channel_name {nptel_channel_name}')
    chanel_vedio_dir = os.path.join(full_vedios_dir, nptel_channel_name)
    playlist_path_list = [x[0] for x in os.walk(chanel_vedio_dir)][1:]

    print(f'number of playlist: {len(playlist_path_list)}')
    vedio_ext = '.mp4'
    
    facef=0
    namef=0
    namenf=0
    facef_playlist_names_list=[]
    namef_playlist_names_list=[]
    namenf_playlist_names_list=[]
    for playlist_path in playlist_path_list[:num_playlists]:
        gotname = False
        gotface = False
        playlistn = playlist_path.split('\\')[-1]
        print(f"\npaylist: {playlistn}")
        vedio_path_list = get_extension_files(playlist_path, ext=vedio_ext)
        print(f"number of vedios: {len(vedio_path_list)}")
        
        vst = 5
        search_interval = 20
        vet = vst + search_interval

        num_f_per_sec = 1
        num_f_inc = 3
        

        text_sub = get_prof_name(playlistn.lower(), title=True)
        if text_sub is not None:
            print(f'name: {text_sub} in the title')
            prof_name = text_sub 
            gotname = True

        for vedio_path in vedio_path_list[:num_vedios]:
            if gotface:
                continue
            else:
                if gotname:
    #                 vst+=search_interval
    #                 vet+=search_interval
                    num_f_per_sec+=num_f_inc
                else:
                    num_f_per_sec+=num_f_inc

            vfname = vedio_path.split('\\')[-1]
            print(f"vedio: {vfname}")
            print(f"(vst, vet, num_f_per_sec): {(vst, vet, num_f_per_sec)}")
            video_capture = cv2.VideoCapture(vedio_path)
            fps = int(video_capture.get(cv2.CAP_PROP_FPS))
            st = tt()
            i=-1
            while video_capture.isOpened():
                i+=1
                if int(i/fps) < vst:
                    ret, frame = video_capture.read()
                    continue

                if int(i/fps) > vet:
                    break

                ret, frame = video_capture.read()

                if i%int(fps/num_f_per_sec)==0:
    #                 print(f"time(sec): {int(i/fps)}")
                    if not gotname:
                        text = get_name_fom_image(frame)
                        if text is not None:
                            print(f'name: {text} at time: {int(i/fps)}')
                            prof_name = text
                            gotname = True

                    else:
                        if not is_valid_img(frame):
                            continue
                        face_locations = face_recognition.face_locations(frame)
                        if len(face_locations)==1:
                            face_tup = face_locations[0]
                            img = frame[face_tup[0]:face_tup[2], face_tup[3]:face_tup[1], :]
                            
                            # check if encoding is empty
                            enc = face_recognition.face_encodings(img)
                            if enc==[]:
                                continue
                                
                            image_path = os.path.join(known_faces_dir, prof_name+'.jpg')
                            if os.path.exists(image_path):
                                print(f"pic for {prof_name} exists ")
                                gotface = True
                                break
                                
                            img = post_processing(img)
                            cv2.imwrite(image_path, img) 
                            gotface = True
                            print(f'face at time: {int(i/fps)}')
                            break

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            video_capture.release()
            cv2.destroyAllWindows()

        if gotface:
            facef+=1
            facef_playlist_names_list.append(playlistn)
            playlist_2_prof_name_dict[playlistn] = prof_name
            
        else:
            if gotname:
                print(f"face not found for {prof_name}")
                namef+=1
                namef_playlist_names_list.append(playlistn)
                playlist_2_prof_name_dict[playlistn] = prof_name
            else:
                print(f"face and name not found for {playlist_path}")
                playlist_2_prof_name_dict[playlistn] = None
                namenf_playlist_names_list.append(playlistn)
                namenf+=1
    
    tfacef+=facef
    tnamef+=namef
    tnamenf+=namenf
    print(f"\nfor channel: {nptel_channel_name}  faces found: {facef}, names found: {namef}, names not found: {namenf}")    

    chanel_pkl_files_dir = os.path.join(pkl_files_dir, nptel_channel_name)
    save_as_pickle(os.path.join(chanel_pkl_files_dir, "facef_playlist_names_list.pkl"), facef_playlist_names_list)
    save_as_pickle(os.path.join(chanel_pkl_files_dir, "namef_playlist_names_list.pkl"), namef_playlist_names_list)
    save_as_pickle(os.path.join(chanel_pkl_files_dir, "namenf_playlist_names_list.pkl"), namenf_playlist_names_list)

print(f"\ntotal faces found: {tfacef}, names found: {tnamef}, names not found: {tnamenf}")
prof_name_2_playlist_dict = {v: k for k, v in playlist_2_prof_name_dict.items()}
save_as_pickle(os.path.join(pkl_files_dir, "prof_name_2_playlist_dict.pkl"), prof_name_2_playlist_dict)
save_as_pickle(os.path.join(pkl_files_dir, "playlist_2_prof_name_dict.pkl"), playlist_2_prof_name_dict)
    
winsound.Beep(freq, duration)

time: 34min 48s


In [26]:
with open(os.path.join(tracebacks_dir, "prof_names_faces.txt"), 'w', encoding='utf-8') as f:
    f.write(cap.stdout)

time: 63.8 ms


In [7]:
# vedio_path = 'C:\\Users\\YASH\\nptel_face_recognition\\full_vedio\\Cloud Computing\\fZ3D6HQrWzs.mp4'
# video_capture = cv2.VideoCapture(vedio_path)
# fps = int(video_capture.get(cv2.CAP_PROP_FPS))
# st = tt()
# i=-1
# while video_capture.isOpened():
#     i+=1
#     if int(i/fps) < 5:
#         ret, frame = video_capture.read()
#         continue

#     if int(i/fps) > 20:
#         break

#     ret, frame = video_capture.read()

#     if i%fps==0:
# #                 print(f"time(sec): {int(i/fps)}")
#         if not gotname:
#             text = get_name_fom_image(frame)
#             if text is not None:
#                 print(f'name: {text} at time: {int(i/fps)}')
#                 prof_name = text
#                 gotname = True

#         else:
#             face_locations = face_recognition.face_locations(frame)
#             if len(face_locations)==1:
#                 face_tup = face_locations[0]
#                 img = frame[face_tup[0]:face_tup[2], face_tup[3]:face_tup[1], :]
#                 image_path = os.path.join(known_faces_dir, prof_name+'.jpg')
#                 cv2.imwrite(image_path, img) 
#                 gotface = True
#                 print(f'face at time: {int(i/fps)}')
#                 break

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# video_capture.release()
# cv2.destroyAllWindows()

time: 9.97 ms
